## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster
from countpassenger import Utils
from countpassenger import Snapshot
from countpassenger import Dataset

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option("display.precision", 10)
start = 1714284000
stop = 1714288000

CURRENT_CAM = "mbk-14-14"
SAVE_DIR = os.path.join(
    conf.RESOURCES_PROCESSED_DIR, f"2024-04-29/count-passengers-{CURRENT_CAM}-vehicle-snapshot.csv"
)

load_param = dict(
    vehicle_path_rel="2024-04-29/mbk-tourist-vehicle-object-20240429.csv",
    cross_path_rel="2024-04-29/mbk-tourist-raw-cross-object-20240429.csv",
    reverse_path_rel="2024-04-29/mbk-tourist-raw-reverse-object-20240429.csv",
)

In [2]:
df_vehicle, df_cross, df_reverse = Dataset.load_dataset_from_paths(**load_param)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Cluster only large vehicle 

In [3]:
params = dict(
    min_cluster_size=2,  # have at least 5 people together
    min_samples=2,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    n_jobs=None,
    cluster_selection_method="eom",  # eom with minsample = 3 is kinda good
    store_centers=None,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross, params=params)
print(
    Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string()
)

IndentationError: unexpected indent (125291892.py, line 15)

In [ ]:
params = dict(
    min_cluster_size=2,
    min_samples=None,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    cluster_selection_method="eom",
    store_centers=None,
)

df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse, params=params)
print(
    Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[
        (cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)
    ][conf.CLUSTER_INTEREST].to_string()
)

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


Empty DataFrame
Columns: [timestamp_min, timestamp_max, count, cluster_id, xmid_mean, ymid_mean]
Index: []


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_36727/215765330.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[


lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

nearest x,y ? in time windows?

In [ ]:
df_vehicle_snapshot_with_c = Snapshot.match_cross_cluster_snapshot_to_vehicle(
    df_vehicle_snapshot=df_vehicle, cluster_cross=cluster_cross, capture_window=70
)
df_vehicle_snapshot_with_c_and_r = Snapshot.match_reverse_cluster_snapshot_to_vehicle(
    df_vehicle_snapshot=df_vehicle_snapshot_with_c, cluster_reverse=cluster_reverse, capture_window=70
)

print(df_vehicle_snapshot_with_c_and_r[conf.VEHICLE_INTEREST_SNAPSHOT_CLUSTERED].to_string())

#Exporting as csv
df_vehicle_snapshot_with_c_and_r.to_csv(path_or_buf=SAVE_DIR, index=False)
# johnny.to_csv(path_or_buf=SAVE_DIR, index = False,date_format=str)

                          camera       timestamp_precise  cross_count  reverse_count vehicle_type              cluster_cross_list cluster_reverse_list plate_number    xmid    ymid
1564  mbk-14-14-vehicle-snapshot 2024-04-29 02:00:37.635            0              0          bus                              []                   []          NaN  1078.5   597.0
1565  mbk-14-14-vehicle-snapshot 2024-04-29 02:01:52.694            0              0        other                              []                   []          NaN  1078.5   596.0
1566  mbk-14-14-vehicle-snapshot 2024-04-29 02:03:07.673            0              0          bus                              []                   []          NaN  1078.5   594.5
1567  mbk-14-14-vehicle-snapshot 2024-04-29 02:04:22.412            0              0          bus                              []                   []          NaN  1080.0   595.5
1552  mbk-14-14-vehicle-snapshot 2024-04-29 02:05:37.636            0              0        other   